In [1]:
# MIT image
# Made by : HERO Kwon
# Date : 190108

import os
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
import math
import cmath

# data path
#path_meta = '/home/herokwon/mount_data/Data/Wi-Fi_meta/'
#path_csi = '/home/herokwon/mount_data/Data/Wi-Fi_processed/'
#path_csi_np = '/home/herokwon/mount_data/Data/Wi-Fi_processed_npy/'

# data path_mi
path_csi = 'J:\\Data\\Wi-Fi_processed\\'
path_csi_np = 'J:\\Data\\Wi-Fi_processed_npy\\'
path_meta = 'J:\\Data\\Wi-Fi_meta\\'
path_sc = 'J:\\Data\\Wi-Fi_info\\'

# data info
df_info = pd.read_csv(path_meta+'data_subc_sig_v1.csv')
person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))
csi_time = 15000 #int(np.max(df_info['len']))
# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

# freq BW list
bw_list = pd.read_csv(path_meta+'wifi_f_bw_list.csv')

# avg Array
with open(path_meta + 'dict_avgcsi.pickle','rb') as f:
    dict_avg = pickle.load(f)

In [2]:
# make data generator
def gen_csi(df_info,dict_avg,id_num,len_num):
    for file in np.unique(df_info.id.values):
        
        # Label
        id_key = df_info[df_info.id==file]['id_person'].values[0].astype('int')
        data1_y = dict_id[id_key]
        
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
            
        # subtract avg
        read_avg = dict_avg[id_key][0]
        nonzero_avg = read_avg[np.nonzero(read_avg)].reshape([-1,30,2,3])
        arr_avg = np.mean(nonzero_avg,axis=0)
        data1_diff = data1 - arr_avg #np.diff(data1,axis=0)
        # zero pad
        pad_len = len_num - data1_diff.shape[0]
        data1_pad = np.pad(data1_diff,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)

        # subcarrier info
        data1_sc_df = pd.read_csv(path_sc + file + '_df_sc.csv')
        data1_time = np.diff(data1_sc_df['timestamp_low'])
        data1_time_pad = np.pad(data1_time,((1,pad_len)),'constant',constant_values=0)
        
        
        yield(data1_pad ,data1_y,id_key,data1_time_pad)


gen = gen_csi(df_info,dict_avg,no_classes,csi_time)
target_sig,target_lab,target_id,target_dt = next(gen)

In [3]:
# 3D scan
m,n = 2,3
c =  299792458 # speed of light 
#r = (160 + 160 + 164) * 0.01 # meter
r = 1.64 #meter
d = 45 * 0.01 # meter
ch = 8#3
max_subc = 30

th_range,si_range = (30,30)
sig_mat = np.zeros([csi_time,2*th_range,2*si_range])

In [4]:
# 3D Reconstruction func
from numba import vectorize
@vectorize(['complex64(complex64,float32,float32,int32,int32,\
float32,float32,float32,int32)'], target='cpu')
def Recon3d(sig,theta,sigma,m,n,lam,d,r,c):
    #above_eq1 = 1j * (2*math.pi) * k * r * t / c
    #above_eq1 = 1j * (2*math.pi) * c * dt * 0.000001 / lam
    above_eq1 = 1j * (2*math.pi) * (160 + 160 + 164) * 0.01 / lam
    above_eq2 = 1j * (2*math.pi/lam) * math.sin(theta) * ((n+1)*d*math.cos(sigma) + (m+1)*d*math.sin(sigma))
    eq_res = sig* cmath.exp(above_eq1) * cmath.exp(above_eq2)
    #eq_res = cmath.exp(above_eq1) * cmath.exp(above_eq2)
    return eq_res #math.atan2(eq_res.imag,eq_res.real)


In [5]:
from numba import jit
#@jit
def Calc3d(target_sig,max_subc,theta,sigma,bw_list,ch,d,r,c,m,n):
    m_array = np.concatenate([np.ones(csi_time*n,dtype=np.int32)*0,np.ones(csi_time*n,dtype=np.int32)*1])
    n_array = np.concatenate([np.ones(csi_time*m,dtype=np.int32)*0,np.ones(csi_time*m,dtype=np.int32)*1,np.ones(csi_time*m,dtype=np.int32)*2])
    for subc in range(max_subc): 
        sig1 = np.ascontiguousarray(target_sig[:,subc,:,:].reshape([csi_time*m*n]), dtype=np.complex64)
        lam =  c*0.000001 / bw_list[str(subc)][ch] #wavelength = 300 / frequency in MHz
        sum_eq = np.zeros(csi_time*m*n,dtype=np.complex_)
        sum_eq += Recon3d(sig1,theta,sigma,m_array,n_array,lam,d,r,c)
    sum_reshape = np.sum(sum_eq.reshape([-1,m,n]),axis=(1,2))
    return(np.abs(sum_reshape))


In [6]:
import time
t1 = time.time()
for idx_th,i in enumerate(range(-th_range,th_range)):
    for idx_si,j in enumerate(range(-si_range,si_range)):
        theta = i * (np.radians(30)/2) / th_range
        sigma = j * (np.radians(30)/2) / si_range
        sig_mat[:,idx_th,idx_si] = Calc3d(target_sig,max_subc,theta,sigma,bw_list,ch,d,r,c,m,n)
        
print(time.time()-t1)

1047.0268580913544


In [7]:
sig_mat.shape

(15000, 60, 60)

In [13]:
temp = np.array()
temp.shape
temp

array([[1, 2, 3],
       [4, 5, 6]])

In [14]:
target_sig.shape

(15000, 30, 2, 3)

In [15]:
temp.flatten()

array([1, 2, 3, 4, 5, 6])

In [17]:
aa = np.ones([15000,30,2,3])
aa.shape

(15000, 30, 2, 3)

In [18]:
for i in range(30):
    aa[:,i,:,:] = i

In [27]:
aa1 = aa.flatten()

In [29]:
aa1[:30]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,
        2.,  2.,  2.,  2.,  2.,  3.,  3.,  3.,  3.,  3.,  3.,  4.,  4.,
        4.,  4.,  4.,  4.])

In [30]:
len(aa1)

2700000

In [31]:
csi_time,max_subc,m,n

(15000, 30, 2, 3)

In [46]:
subc_array = np.ones([csi_time,max_subc,m,n],dtype=np.int32)
m_array = np.ones([csi_time,max_subc,m,n],dtype=np.int32)
n_array = np.ones([csi_time,max_subc,m,n],dtype=np.int32)

In [47]:
for i in range(max_subc):
    subc_array[:,i,:,:] = i
for i in range(m):
    m_array[:,:,i,:] = i
for i in range(n):
    n_array[:,:,:,i] = i

In [48]:
subc_array = subc_array.flatten()
m_array = m_array.flatten()
n_array = n_array.flatten()

In [53]:
lam_array = np.array([c*0.000001/bw_list[str(e)][ch] for e in subc_array],dtype=np.float32)

In [54]:
max_subc

30

In [116]:
# 3D Reconstruction func
from numba import vectorize
@vectorize(['complex64(complex64,float32,float32,int32,int32,float32,float32,float32,int32)'], target='cuda')
def Recon3d(sig,theta,sigma,m,n,lam,d,r,c):
    #above_eq1 = 1j * (2*math.pi) * k * r * t / c
    #above_eq1 = 1j * (2*math.pi) * c * dt * 0.000001 / lam
    above_eq1 = 1j * (2*math.pi) * (160 + 160 + 164) * 0.01 / lam
    above_eq2 = 1j * (2*math.pi/lam) * math.sin(theta) * ((n+1)*d*math.cos(sigma) + (m+1)*d*math.sin(sigma))
    eq_res = sig* cmath.exp(above_eq1) * cmath.exp(above_eq2)
    #eq_res = cmath.exp(above_eq1) * cmath.exp(above_eq2)
    return eq_res #math.atan2(eq_res.imag,eq_res.real)

In [117]:
from numba import jit
#@jit
def Calc3d(target_sig,max_subc,theta,sigma,bw_list,ch,d,r,c,m,n):
    #vectorize arrays
    subc_array = np.ones([csi_time,max_subc,m,n])
    m_array = np.ones([csi_time,max_subc,m,n])
    n_array = np.ones([csi_time,max_subc,m,n])
    for i in range(max_subc):
        subc_array[:,i,:,:] = i
    for i in range(m):
        m_array[:,:,i,:] = i
    for i in range(n):
        n_array[:,:,:,i] = i
    subc_array = subc_array.flatten().astype(np.int32)
    m_array = m_array.flatten().astype(np.int32)
    n_array = n_array.flatten().astype(np.int32)
    lam_array = np.array([c*0.000001/bw_list[str(e)][ch] for e in subc_array],dtype=np.float32)
    
    d_array=d*np.ones(csi_time*max_subc*m*n).astype(np.float32)
    r_array=r*np.ones(csi_time*max_subc*m*n).astype(np.float32)
    c_array=c*np.ones(csi_time*max_subc*m*n).astype(np.int32)
    th_array = theta*np.ones(csi_time*max_subc*m*n).astype(np.float32)
    si_array = sigma*np.ones(csi_time*max_subc*m*n).astype(np.float32)
    target_array = target_sig.flatten().astype(np.complex64)
    
    #3d recon
    sum_eq = np.zeros(csi_time*max_subc*m*n,dtype=np.complex_)
    sum_eq = Recon3d(target_array,th_array,si_array,m_array,n_array,lam_array,d_array,r_array,c_array)

    return(sum_eq)
    #sum_reshape = np.sum(sum_eq.reshape([-1,max_subc,m,n]),axis=(1,2,3))
    #return(np.abs(sum_reshape))

In [118]:
for idx_th,i in enumerate(range(-th_range,th_range)):
    for idx_si,j in enumerate(range(-si_range,si_range)):
        theta = i * (np.radians(30)/2) / th_range
        sigma = j * (np.radians(30)/2) / si_range

In [119]:
import time
t1 = time.time()
a = Calc3d(target_sig,max_subc,theta,sigma,bw_list,ch,d,r,c,m,n)
print(time.time()-t1)

31.96660876274109


C:\Users\herok\Anaconda3\envs\mit_image\lib\site-packages\numba\cuda\compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
